(cont:mm:paging)=
# Paging

## Paging - Avoiding Fragmentation

The fragmentation in
[\[fig:vm:fig2\]](#fig:vm:fig2){reference-type="autoref"
reference="fig:vm:fig2"} is termed *external fragmentation*, because the
memory wasted is *external* to the regions allocated. This situation can
be avoided by *compacting* memory---moving existing allocations around,
thereby consolidating multiple blocks of free memory into a single large
chunk. This is a slow process, requiring processes to be paused, large
amounts of memory to be copied, and base+bounds registers modified to
point to new locations[^2].

![image](../images/pb-figures/mm/virt-mem-map.png){height="8\\baselineskip"}

Instead, modern CPUs use *paged address translation*, which divides the
physical and virtual memory spaces into fixed-sized pages, typically
4KB, and provides a flexible mapping between virtual and physical pages,
as shown in [\[fig:vm:fig3\]](#fig:vm:fig3){reference-type="autoref"
reference="fig:vm:fig3"}. The operating system can then maintain a list
of free physical pages, and allocate them as needed. Because any
combination of physical pages may be used for an allocation request,
there is no external fragmentation, and a request will not fail as long
as there are enough free physical pages to fulfill it.

Instead, modern CPUs use *paged address translation*, which divides the
physical and virtual memory spaces into fixed-sized pages, typically
4KB, and provides a flexible mapping between virtual and physical pages,
as shown in [\[fig:vm:fig3\]](#fig:vm:fig3){reference-type="autoref"
reference="fig:vm:fig3"}. The operating system can then maintain a list
of free physical pages, and allocate them as needed. Because any
combination of physical pages may be used for an allocation request,
there is no external fragmentation, and a request will not fail as long
as there are enough free physical pages to fulfill it.

### Internal Fragmentation

Paging solves the problem of external fragmentation, but it suffers from
another issue, *internal fragmentation*, because space may be wasted
*inside* the allocated pages. E.g. if 10 KB of memory is allocated in
4KB pages, 3 pages (a total of 12 KB) are allocated, and 2KB is wasted.
To allocate hundreds of KB in pages of 4KB this is a minor overhead:
about $\frac{1}{2}$ a page, or 2 KB, wasted per allocation. But internal
fragmentation makes this approach inefficient for very small allocations
(e.g. the `new` operator in C++), as shown in
[\[fig:vm:fig4\]](#fig:vm:fig4){reference-type="autoref"
reference="fig:vm:fig4"}. (It is also one reason why even though most
CPUs support multi-megabyte or even multi-gigabyte "huge" pages, which
are slightly more efficient than 4 KB pages, they are rarely used.)

![image](../images/pb-figures/mm/virt-mem-pic7.png){width="90%"}\

## Paged Address Translation

We examine a single model of address translation in detail: the one used
by the original Pentium, and by any Intel-compatible CPU running in
32-bit mode. It uses 32-bit virtual addresses, 32-bit physical
addresses, and a page size of 4096 bytes. Since pages are $2^{12}$ bytes
each, addresses can be divided into 20-bit page numbers and 12-bit
offsets within each page, as shown in
[\[fig:vm:fig5\]](#fig:vm:fig5){reference-type="autoref"
reference="fig:vm:fig5"}


![image](../images/pb-figures/mm/virt-mem-pic10.png){height="5.5\\baselineskip"}


The Memory Management Unit (MMU) maps a 20-bit virtual page number to a
20-bit physical page number; the offset can pass through unchanged, as
shown in [\[fig:vm:fig6\]](#fig:vm:fig6){reference-type="autoref"
reference="fig:vm:fig6"}, giving the physical address the CPU should
access.

![Page number and offset in 32-bit paged translation with 4KB
pages](../images/pb-figures/mm/virt-mem-pic9.png){#fig:vm:fig5 width="\\textwidth"}

Although paged address translation is far more flexible than base and
bounds registers, it requires much more information. Base and bounds
translation only requires two values, which can easily be held in
registers in the MMU. In contrast, paged translation must be able to
handle a separate mapping value for each of over a million virtual
pages. (although most programs will only map a fraction of those pages)
The only possible place to store the amount of information required by
paged address translation is in memory itself, so the MMU uses page
tables in memory to specify virtual-to-physical mappings.